# Word Transcript Tabulator Recipe

This notebook is a computational recipe to help you to take a collection of transcripts in word documents (_docx_ only) and turn them into a structured tabular format suitable for working with computational analytics approaches.

It will also:

1. Help you check for and identify some inconsistencies in your transcripts (for example, missing and inconsistent speaker codes).
2. Prepare a spreadsheet description of your dataset that can be converted to a standard metadata format (RO-Crate).

## Assumptions and Preparation:

- You have a set of transcripts as Microsoft Word (`.docx`) files.
- Transcripts are formatted as one line/paragraph per speaker turn.
- There is one speaker per turn, indicated by a speaker code at the start of the turn and ending with a colon (`:`) character.
- Blank lines can be ignored.
- No formatting of text is used to indicate important information: this process will remove styling information like *bold* and _italics_.
- This won't be perfect: be prepared to spend time identifying and fixing errors.

## This isn't for you if:

1. You don't have your transcripts in Word format.
2. You have formatted your transcripts as tables in Word.


## Workflow Overview

1. Upload your transcripts in `.docx` format to the transcripts folder.
2. Run the script to produce an initial file for examination and manual changes.
4. Use the produced metadata to identify and fix inconsistencies in the transcripts *in word* - reupload any changed versions and try again.
5. Fill in the metadata for speaker_codes and speakers. (Match codes in transcripts to speaker_ids, give details known about speakers).


# Getting Started

1. Upload your transcripts in `.docx` format into the `transcripts` folder on the left - you might might need to expand the left hand tab by clicking on the `icon of a folder`. Nested files are currently not supported.
2. Run the code blocks below one at a time - you can run a code block by placing your cursor (clicking into) that cell and pressing the play button or entering the keyboard shortcut `shift-enter`.


In [1]:
# Load the libraries we're going to use
library(officer)
library(writexl)
library(dplyr)
library(stringr)

# Define some helper functions: these are how we extract individual units of data from your transcripts

# Do the initial extraction from a particular file
prepare_transcripts <- function(transcript_file_path, speaker_regex, remove_speaker_regex){
    print(transcript_file_path)
    
    # Load the file from word
    transcript_df <- officer::docx_summary(officer::read_docx(transcript_file_path))
    
    # Remove any lines that contain only whitespace
    transcript_df <- transcript_df %>%
        filter(trimws(transcript_df$text) != '')
    
    # Attach the filepath as a column so we can trace this back
    transcript_df$source_file <- basename(transcript_file_path)
    
    # Keep a copy of the original text so we can always compare the processing
    transcript_df$source_text <- transcript_df$text
    
    # Extract the speaker codes via regex - match up to, but don't include the first colon.
    transcript_df$speaker_code <- stringr::str_extract(transcript_df$source_text, speaker_regex)
    
    # And replace the original text with the speaker code removed
    transcript_df$text <- NULL
    transcript_df$text <- stringr::str_replace(transcript_df$source_text, remove_speaker_regex, "")    

    return(transcript_df)
}


Warning message:
“package ‘writexl’ was built under R version 4.4.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Extract Turns and Speaker Information

This next block of code will load all of the transcript files you have uploaded and:

1. Remove the headers.
2. Extract the individual turns of speakers as rows.
3. Join lines with leading whitespace (turns with manual line breaks) into single turns

In [2]:
# Now let's actually load your transcripts

# List all of the docx files in the transcripts folder
transcript_files = list.files("transcripts", full.names=TRUE, pattern="*.docx")

paste0("Loading: ", length(transcript_files), " transcripts:") 

# The speaker code matches: a block of alphanumeric characters from the start of the turn,
# up to, but not including the first colon. This will break if:
# - the speaker code has a space
speaker_regex <- '^[[:alpha:]]+?(?=:)'
# This will match the speaker code, and the colon character, and any whitespace characters following.
remove_speaker_regex <- '^[[:alpha:]]+?:[[:space:]]*'

# TODO: figure out how to handle warnings without scaring people and ignoring them...
# Load each transcript, extract the paragraphs of the document as rows in a dataframe, using the officer package
loaded_docs <- lapply(transcript_files, prepare_transcripts, speaker_regex=speaker_regex, remove_speaker_regex=remove_speaker_regex)

# Combine the transcripts together into a single dataframe.
combined_transcripts <- bind_rows(loaded_docs)

[1] "Loading: 6 transcripts:"

[1] "transcripts/Transcrp-BethDaniel.docx"
[1] "transcripts/Transcrp-HeatherMarie.docx"
[1] "transcripts/Transcrp-LisaFiona.docx"
[1] "transcripts/Transcrp-MarkKylie.docx"
[1] "transcripts/Transcrp-NatalieKen.docx"
[1] "transcripts/Transcrp-SuzanneLen.docx"


In [3]:
# Now prepare three views of this dataset, and combine them with the existing data in the spreadsheet.
# Merge or replace test?

# Summary of the files and number of turns
transcript_turns <- combined_transcripts %>% 
    group_by(source_file) %>% 
    summarise(turns=n(), speaker_count=n_distinct(speaker_code))

# Info about the speaker-codes in each transcript
speaker_code_summary <- combined_transcripts %>% 
    group_by(source_file, speaker_code) %>%
    summarise(n_turns=n(), first_turn=min(doc_index), last_turn=max(doc_index), .groups="drop_last")

# Select and order columns for display
output_turns <- combined_transcripts %>% 
    select(speaker_code, text, source_file, source_text, doc_index)

# An (initially empty) table for speaker information
sheets <- list(
    transcripts = transcript_turns, 
    turns = output_turns, 
    speaker_codes = speaker_code_summary
)

write_xlsx(
    sheets,
    path='combined_transcripts.xlsx'
)

IRdisplay::display_html(
    '<a href="../combined_transcripts.xlsx" download><h2>Download your combined transcripts.</h2></a>'
)


Download your combined transcripts.